In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.chat_message_histories import ChatMessageHistory

from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('GOOGLE_API_KEY')

In [2]:
# Google gemini for chat based interaction
chat_model = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.2, google_api_key=API_KEY,
                                    convert_system_message_to_human=True)

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

# Prompt template for guiding the all chat conversation
system_message = """You are a helpful assistant. Answer all questions to the best of your ability with detailed and 
                    actionable response."""
system_prompt = ChatPromptTemplate.from_messages([('system', system_message), 
                                           MessagesPlaceholder(variable_name='messages')])

# Chain the model with the 
chain = system_prompt | chat_model

The `MessagesPlaceholder` above inserts chat messages passed into the chain’s input.

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

prompt = 'Translate this sentence from English to French: I love programming.'
chain.invoke(
    {'messages': [HumanMessage(content=prompt)]}
)

AIMessage(content=" J'aime la programmation.")

That look greats, however, the model itself doesn't have the ability to sustain a normal conversation. Just see that,

In [5]:
chain.invoke({'messages': [HumanMessage(content='What did you just say')]})

AIMessage(content='I am a helpful assistant designed to provide comprehensive and actionable responses to your questions. I strive to deliver accurate and informative answers to the best of my abilities.\n\nIf you have any specific questions or need assistance with a particular topic, please feel free to ask, and I will do my best to provide you with a detailed and helpful response.')

You can see that he doesn't have any conception about the last conversation, and seem to be lost.

Having a message history will fix this issue and allow him to track the previous conversation through history.    

# Memory without RAG

In [6]:
# Based on preference
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.schema import SystemMessage


In [7]:
# Prompt template for guiding the all chat conversation
system_message = "You are a helpful assistant. Answer all questions to the best of your ability."
system_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_message), 
        MessagesPlaceholder(variable_name='chat_history'),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ])

In [8]:
from langchain.chains import LLMChain
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation = LLMChain(llm=chat_model, prompt=system_prompt, memory=memory, verbose=False)

In [9]:
prompt = 'Translate this sentence from English to French: I love programming.'
conversation.predict(human_input=prompt)

"J'adore la programmation."

In [10]:
conversation.predict(human_input='What did you just say')

'I just said "I love programming" in French.\n\nThe original sentence in English was "I love programming." I used Google Translate to translate this sentence into French, which gave me the translation "J\'adore la programmation."\n\nIs there anything else I can help you with today?'

That look awesome, we have just introduce a memory to the chatbot. Now we can have a great conversation with it which allow us to use it in various application like assistant, client support or ecommerce. Also, we can use it by adding external data source to interact with which bring us to the next step **Retrieval**.

# Retrieval

In this section, we will use Milvus as VectorStore to store all the information that the chatbot through all of the conversation. 

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Milvus
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pymilvus import connections, db, utility

In [12]:
# Load new data
loader = PyPDFLoader("Selling_the_Invisible_A_Field_Guide_to_M.pdf")
docs = loader.load()

In [13]:
# Embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)

# Document Splitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
docs_splitted = text_splitter.split_documents(docs)

# Vectore store
connection_args = {"host": "127.0.0.1", "port": "19530"}
vector_store = Milvus(embeddings, connection_args=connection_args, 
                      collection_name='Selling_Invisible')

# k is the number of chunks to retrieve
retriever = vector_store.as_retriever()

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.

<context>
{context}
</context>
""" 

question_answering_prompt = ChatPromptTemplate.from_messages(
    [("system", SYSTEM_TEMPLATE), MessagesPlaceholder(variable_name="messages")]
)
document_chain = create_stuff_documents_chain(chat_model, question_answering_prompt)

In [15]:
document_chain.invoke({
    "context": retriever.invoke("How to sell a service ?"),
    "messages": [HumanMessage(content="How to sell a service ?")]
})

'- Understand and satisfy the person before trying to satisfy the client.\n- Make every employee a marketer.\n- Create the possible service, not just what the market needs or wants.'

Assembling stuff.

In [16]:
from langchain_core.runnables import RunnablePassthrough
from typing import Dict

def parse_retriever_input(params: Dict):
    return params["messages"][-1].content

retrieval_chains = RunnablePassthrough.assign(context=parse_retriever_input | retriever,).assign(answer=document_chain)

In [17]:
retrieval_chains.invoke({
    "messages": [HumanMessage(content="How to sell a service ?")]
})

{'messages': [HumanMessage(content='How to sell a service ?')],
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a service, you’re  actually selling a relationship. \n \nBefore you try to satisfy your client, understand and satisfy the person. That means knowing his \nlikes, hobbies, pets, family life, dreams and goals. \n \nYour real competition is sitting across the table from you. Plan accordingly. \n \nYou

In [18]:
retriever.invoke("Tell me more!")

[Document(page_content='Last word: Take the risk. Get out there and let opportunity hit you.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 9}),
 Document(page_content='subscriptions if reply forms are ready and postage has been paid for. Forget looking \nlike the superior choice, focus on being an ex cellent choice. Eliminate anything that \nmakes you a bad choice.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 5}),
 Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Fiel

## Query transformation

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

QUERY_SYSTEM_TEMPLATE = """Answer the user's questions based on the below context. """ 
query_transform_prompt = ChatPromptTemplate.from_messages([
    ("system", QUERY_SYSTEM_TEMPLATE),
    MessagesPlaceholder(variable_name="messages")
])

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        (lambda x: x["messages"][-1].content) | retriever
    ),
    query_transform_prompt | chat_model | StrOutputParser() | retriever
).with_config(run_name="chat_retriever_chain")

In [20]:
query_transform_chain = query_transform_prompt | chat_model

query_transform_chain.invoke({
    "messages": [
        HumanMessage(content="How to sell a service ?"),
        AIMessage(content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business.'),
        HumanMessage(content="Summarize!")
    ]
})

AIMessage(content="To sell a service, focus on the customer's needs and make it easy for them to buy from you.")

In [21]:
conversation_retriever_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain
).assign(answer=document_chain)

In [22]:
conversation_retriever_chain.invoke(
    {
        "messages": [HumanMessage(content="How to sell a service ?")]
    }
)

{'messages': [HumanMessage(content='How to sell a service ?')],
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a service, you’re  actually selling a relationship. \n \nBefore you try to satisfy your client, understand and satisfy the person. That means knowing his \nlikes, hobbies, pets, family life, dreams and goals. \n \nYour real competition is sitting across the table from you. Plan accordingly. \n \nYou

In [23]:
conversation_retriever_chain.invoke(
    {
        "messages": [
            HumanMessage(content="How to sell a service ?"),
            AIMessage(
                content="- Tell people in a single sentence why they should buy from you.\n- Advertise.\n- Make your service easy to order or buy.\n- Talk about the other person, not yourself.\n- Show that you care passionately about your clients’ business."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)

{'messages': [HumanMessage(content='How to sell a service ?'),
  AIMessage(content='- Tell people in a single sentence why they should buy from you.\n- Advertise.\n- Make your service easy to order or buy.\n- Talk about the other person, not yourself.\n- Show that you care passionately about your clients’ business.'),
  HumanMessage(content='Tell me more!')],
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a 

## Add memory to query transformation

In [38]:
chat_history = ChatMessageHistory()
ASSISTANT_PROMPT = "You're a helpful assistant that help human to reach their goal"
query_conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", QUERY_SYSTEM_TEMPLATE),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

query_transforming_chain = query_conversation_prompt | chat_model
query_transforming_chain_with_history = RunnableWithMessageHistory(
    query_transforming_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [39]:
query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("chat_history", [])) == 1,
        (lambda x: x["chat_history"][-1].content) | retriever
    ),
    query_transforming_chain_with_history | StrOutputParser() | retriever
).with_config(run_name="chat_mode")

In [40]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context.

<context>
{context}
</context>
""" 

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_TEMPLATE), 
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

document_chain = question_answering_prompt | chat_model

# Add history
document_chain_with_history = RunnableWithMessageHistory(
    document_chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [41]:
conversation_retriever_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain
).assign(answer=document_chain_with_history)

In [42]:
response = conversation_retriever_chain.invoke(
    {
        "input": "How to use social proof for selling my stuff ?"
    },
    {"configurable": {"session_id": "unused"}}
)
print(response['answer'].content)

**How to Use Social Proof for Selling Your Stuff**

Social proof is a psychological phenomenon where people conform to the actions or beliefs of others. In the context of selling, social proof can be used to persuade potential customers to buy your products or services by showing them that others have already done so.

Here are some ways to use social proof for selling your stuff:

* **Display customer testimonials and reviews:** Positive feedback from satisfied customers can be a powerful form of social proof. Display testimonials and reviews on your website, social media pages, and other marketing materials.
* **Use social media to show how others are using your products:** Share photos and videos of people using and enjoying your products. This can help potential customers see how your products can fit into their own lives.
* **Partner with influencers:** Partner with influencers in your industry to promote your products. When influencers endorse your products, it can give them a se

In [43]:
response = conversation_retriever_chain.invoke(
    {
        "input": "Could you provide an example ?"
    },
    {"configurable": {"session_id": "unused"}}
)
print(response['answer'].content)

**Example of Using Social Proof for Selling a Service**

Let's say you are a freelance writer and you want to use social proof to sell your services. Here are some things you could do:

* **Display testimonials from satisfied clients on your website and social media pages:** Include positive feedback from clients who have been happy with your writing services. This will show potential clients that others have had a good experience working with you.
* **Use social media to share examples of your work:** Share blog posts, articles, and other writing samples that you have created for clients. This will help potential clients see the quality of your work and how it can benefit them.
* **Partner with other businesses in your industry:** Partner with complementary businesses, such as web designers or marketing agencies. Offer to provide your writing services to their clients, and they can offer their services to your clients. This can help you reach a wider audience and build credibility.
* 

In [46]:
response

{'input': 'Could you provide an example ?',
 'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='examples to make your point. Never use two words when one will do.  \n• Prove your service quality. Be hones t. Tricks and gimmicks are not the key. \n• If you think your promotional idea is silly or unprofessional, it is. \n• Prospects buy how good you are at who you are. \n• Convey that you are positively good. Do not try to say you are the best or superior. \nPeopl

# Add memory management to RAG

## RAG without Memory

In [30]:
from langchain import hub


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [("system", SYSTEM_TEMPLATE), MessagesPlaceholder(variable_name="messages")]
)
prompt = """Use the following context to answer the question at the end.
    Always say “Thanks for asking!” » at the end of the answer.

    {context}
    """

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)
question_answering_prompt = ChatPromptTemplate.from_messages([
    ("system", prompt),
    # MessagesPlaceholder(variable_name="messages")
    ("human", "{question}")
])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | question_answering_prompt
    | chat_model
    | StrOutputParser()
)

In [31]:
rag_chain.invoke("How to sell a service ?")

'You’re actually selling a relationship when you’re selling a service. Thanks for asking!'

Show source that is used to understand well the response.  

In [32]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x['context'])))
    | QA_CHAIN_PROMPT
    | chat_model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [33]:
rag_chain_with_source.invoke("How to sell a service ?")

{'context': [Document(page_content='• Tell people in a single sentence why they should buy from you. \n• Advertise. \n• Make your service easy to order or buy. \n• Talk about the other person, not yourself. \n• Show that you care passionately about your clients’ business. \n• Write a mission statement and keep it private. \n• Draw a clear map. After every missi on statement, add an objectives statement. \n• If the mission statement doesn’t inspire people to act, change it.', metadata={'source': 'Selling_the_Invisible_A_Field_Guide_to_M.pdf', 'page': 8}),
  Document(page_content='are buying an experience. \n \nIf you are selling a service, you’re  actually selling a relationship. \n \nBefore you try to satisfy your client, understand and satisfy the person. That means knowing his \nlikes, hobbies, pets, family life, dreams and goals. \n \nYour real competition is sitting across the table from you. Plan accordingly. \n \nYour prospect faces three options: using your servic e, doing it th

## Add chat history

In [34]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_q_chain = contextualize_q_prompt | chat_model | StrOutputParser()

In [35]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input['question']

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualized_question | retriever | format_docs
    )
    # | RunnablePassthrough.assign(context=(lambda x: format_docs(x["chat_history"])))
    | qa_prompt
    | chat_model
)

In [36]:
chat_history = []
question = "How to sell a service ?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])

second_question = "Provide an example !"
rag_chain.invoke({"question": second_question, "chat_history": chat_history})

AIMessage(content='**Example:**\n\n"Our personalized financial planning service is designed to help you achieve your financial goals, no matter how complex. We take the time to understand your unique needs and aspirations, and create a tailored plan that empowers you to make informed decisions about your financial future."')

In [37]:
chat_history

[HumanMessage(content='How to sell a service ?'),
 AIMessage(content="To sell a service effectively, it's crucial to focus on building a relationship with the client by understanding their needs and aspirations. By creating a service that goes beyond their expectations and making it easy to purchase, you can differentiate yourself from competitors and establish a loyal customer base.")]